<div>
<div style="text-align:center; display:block; float:left; padding:80px;"><img width="200px"  src="https://kaggle2.blob.core.windows.net/competitions/kaggle/4651/logos/front_page.png"/><span style="color:red;">**New User Booking**</span></div>
<div style="">
**Objective:** In this recruiting competition, Airbnb challenges you to predict in which country a new user will make his or her first booking.  
  
** Description: ** In this challenge, you are given a list of users along with their demographics, web session records, and some summary statistics. You are asked to predict which country a new user's first booking destination will be. All the users in this dataset are from the USA.
</div>
<img src="https://kaggle2.blob.core.windows.net/competitions/kaggle/4651/media/airbnb_banner.png" />

Author: [Oliveira, D. M.](http://br.linkedin.com/in/dmoliveira)

## <span style="color:blue">Airbnb - New User Booking - v5 Extreme Learning Machines
Research source: http://bit.ly/1Sq97Xy

## Import Packages

In [2]:
using ELM
using DataFrames

## Load Data

In [3]:
train = readtable("data/train_v2.tsv", separator='\t')
test  = readtable("data/test_v2.tsv", separator='\t')
full = vcat(train, test);

In [4]:
label    = :country_destination
features = setdiff(names(test), [:id]);

## Prepare Data

In [14]:
function split_train_val(df; train_size=.85, random_state=1)
    
    srand(random_state)
    
    nrows, ntraining_rows = size(df, 1), round(Int, size(df, 1) * train_size)
    indexes               = shuffle(collect(1:nrows))
    train                 = df[indexes[1:ntraining_rows], :]
    validation            = df[indexes[ntraining_rows+1:end], :]
    
    return train, validation
end

split_train_val (generic function with 1 method)

In [15]:
train[label]  -= 1
X_train, X_val = split_train_val(train, train_size=.85, random_state=1)

train_x = Array{Float64,2}(X_train[:, features])
train_y = Array{Float64,1}(X_train[label])
val_x   = Array{Float64,2}(X_val[:, features])
val_y   = Array{Float64,1}(X_val[label])
test_x  = Array{Float64,2}(test[:, features]);

In [16]:
nrows, ncols = size(train_x);

## Train

In [25]:
elm = ExtremeLearningMachine(2ncols)
ELM.fit!(elm, train_x, train_y);

## Evaluate

In [24]:
eval_rmse(y, yhats) = round(sqrt(sum((y - yhats) .^ 2)/length(y)), 4)
eval_precision(y, yhats) = round(sum(Array{Int,1}(y .== yhats))/length(y), 4)

eval_precision (generic function with 1 method)

In [26]:
yhat_train = round(ELM.predict(elm, train_x))
yhat_val   = round(ELM.predict(elm, val_x));

In [27]:
println("Eval Train - RMSE: ", eval_rmse(train_y, yhat_train), " Precision: ", eval_precision(train_y, yhat_train))
println("Eval Val   - RMSE: ", eval_rmse(val_y, yhat_val), " Precision: ", eval_precision(val_y, yhat_val))

Eval Train - RMSE: 1.608 Precision: 0.4137
Eval Val   - RMSE: 1.619 Precision: 0.412


## Export Model